# Project Title: Data Engineering Capstone Project
## Author: Raúl Antón Cuadrado.

Contact me at 
- raulanton@gmail.com
- https://remotefrog.com/hello or https://remotefrog.com/hola

## Project Summary

*The project lives in a Github repo https://github.com/rantoncuadrado/udacity_capstone_project/
*The project scope and definitions are in https://github.com/rantoncuadrado/udacity_capstone_project/blob/main/scope.md
*The datasources detailed definition are in https://github.com/rantoncuadrado/udacity_capstone_project/blob/main/Datasources%20Description.md


In [1]:
## NOTES on installing a pip package in the current Jupyter kernel
#import sys
#!{sys.executable} -m pip install s3fs
#!{sys.executable} -m pip install boto
#!{sys.executable} -m pip install boto3
#!{sys.executable} -m pip install pyspark

In [1]:
## IMPORTS 

import pandas as pd

from datetime import datetime

from s3_local_io import *
from create_parquet_tables import *
from data_quality_checks import *

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, count, lit, when, max, lower, countDistinct


In [2]:
## Global DEFINITIONS

# URL and PATHS to data
bucket_name = 'raul-udacity'
bucket_parquet_path ='/parquet/'
bucket_path = 's3a://'+bucket_name+'/'
local_path = './input_files/'
local_parquet_path = './input_files/parquet_files/'
# For context s3a vs s3 explanation https://stackoverflow.com/questions/33356041/technically-what-is-the-difference-between-s3n-s3a-and-s3

# FILENAMES
data_bares = 'bares.csv'
data_restaurantes = 'restaurantes.csv'
data_cafeterias = 'cafeterias.csv'

data_asociaciones = 'AsociacionesJCyL.csv'
data_clubes_deportivos = 'Clubes deportivos.csv'

data_bibliotecas = 'Directorio de Bibliotecas de Castilla y León.json'
data_museos = 'Directorio de Museos de Castilla y León.json'

data_poblacion = 'Cities population per gender age.csv'

# Other available data/filenames we decided not to use
# Poblacion municipio sexo relacion nacimiento residencia.json
# Municipios Origen Nacimiento.csv

# 1.- DATA SOURCES PREPARATION


## 1.1.- Copy Files from S3 to local filesystem

There are 2 options for the data sources (csv and json files) 
1) We can use files in S3
2) We can work on local files (This is the default option to avoid s3 costs). IN this case, we need to copy s3 files to local.

In [4]:
## COPY FILES FROM s3 TO LOCAL
## This step is not needed if working with s3 files
## This step is not needed if they are already copied.

# copy_files_s3_to_local(bucket_name, local_path)

## 1.2.- Creation of the Spark Session

I'll use Spark dataframes to gather and clean the data and later to save the results in parquet files
It would work both over s3 or in local filesystem

In [3]:
# Create an SPARK SESSION 

spark_session = SparkSession \
        .builder \
        .appName("Castilla y Leon -> Fact Tables") \
        .getOrCreate()


# This is needed just if we use spark on s3
#spark_session.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider","org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
#spark_session.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key",os.environ['AWS_ACCESS_KEY_ID'])
#spark_session.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key",os.environ['AWS_SECRET_ACCESS_KEY'])


# 2.- GARITOS TABLE
Garitos is the spanish word I used to comprise bars, restaurants, cafés and maybe other kind of related businesses in the future. 

## 2.1.- Gather Bars table, Restaurants table, Cafes table and puting them together into a SparkDF Garitos.
These 3 files share same schema

In [4]:
sparkdf_garitos=create_garitos(spark_session,local_path,[data_bares,data_restaurantes,data_cafeterias])

# 3.- CITY - POSTAL CODE TABLE.  

We suppose city/postal code table is not available and we infer it from the info we have in garitos table 
Goal: we'll make a parquet table available.


In [6]:
sparkdf_postal_codes=create_postal_code(sparkdf_garitos)

In [7]:
df=sparkdf_postal_codes.toPandas()
df.describe(include='all')

,county,city,postal_code
count,2881,2881,2881
unique,9,1730,2025
top,León,Zamora,24000
freq,519,33,23


In [7]:
toparquet_postal_codes(spark_session,local_parquet_path,sparkdf_postal_codes)

## 2.2.- Garitos table Quality Check and Wrangle/Cleanup

### 2.2.1.- Garitos table. Check for null values in key columns

In [18]:
##CHECK FOR NULL VALUES IN KEY COLUMNS
check_sparkdf_not_nulls(sparkdf_garitos,['name','county','city','garito_kind'])

Checking DataFrame. No null values found in column name
Checking DataFrame. No null values found in column county
Checking DataFrame. No null values found in column city
Checking DataFrame. No null values found in column garito_kind


True

### 2.2.2.- Garitos table. Check for duplicate values.

In [11]:
##WE SHOULD NOT CHECK FOR DUPES AS DUPES ARE PERFECTLY OK
## THey mean several bar/restaurant/cafes with the same name in a given city
## This happens with Burguer King and similars. 
## An example here:

a=sparkdf_garitos.select("*").groupBy('name','county','city','garito_kind') \
    .agg(count("name").alias("Total")) \
    .orderBy("Total", ascending=False)

a.head(10)

[Row(name='BURGER KING', county='Valladolid', city='Valladolid', garito_kind='restaurantes', Total=8),
 Row(name='TELEPIZZA', county='Valladolid', city='Valladolid', garito_kind='bares', Total=6),
 Row(name='LA BODEGUILLA', county='León', city='León', garito_kind='bares', Total=5),
 Row(name='GARCIA', county='León', city='León', garito_kind='bares', Total=5),
 Row(name='EL PASO', county='Valladolid', city='Valladolid', garito_kind='bares', Total=4),
 Row(name='TELEPIZZA', county='Burgos', city='Burgos', garito_kind='bares', Total=4),
 Row(name='BERTIZ', county='Burgos', city='Burgos', garito_kind='cafeterias', Total=4),
 Row(name='BURGER KING', county='León', city='León', garito_kind='restaurantes', Total=4),
 Row(name='PAN EL VISO', county='Zamora', city='Zamora', garito_kind='cafeterias', Total=4),
 Row(name='BURGER KING', county='Burgos', city='Burgos', garito_kind='restaurantes', Total=4)]

### 2.2.3.- Garitos table Wrangling: we can have null postal codes, but we'll try to guess and fill them when possible.

In [36]:
## GUESS NULL POSTAL CODE WHEN POSSIBLE AND COMPLETE THESE ROWS

# We could have rows with null postal codes. IN these cases we'll try to find the one corresponding with
# the county and city in the postal code table (when there is just one for that city) and
# we'll complete the row with the postal code

check_sparkdf_not_nulls(sparkdf_garitos,['postal_code'])

Checking DataFrame. I found null values in column postal_code


False

In [37]:
# We found rows with null postal_code so we put all these rows in a spark dataframe

sparkdf_garitos_pc_null=sparkdf_garitos.select(
            '*'
            ).where(col('postal_code').isNull())

sparkdf_garitos=sparkdf_garitos.select(
            '*'
            ).where(col('postal_code').isNotNull())

sparkdf_garitos_pc_null.head(2)

[Row(name='PUB EBANO', address='C/ JUAN FERRERO Nº 80', county='León', city='Valderrueda', postal_code=None, garito_kind='bares'),
 Row(name='SILVAN', address=None, county='León', city='Torre del Bierzo', postal_code=None, garito_kind='bares')]

In [38]:
# We extract the list of cities with unique postal code (The only ones we can use to guess postal codes)

cities_with_unique_postal_codes=sparkdf_postal_codes.select(
            'city',
            'postal_code'
            ).groupBy("city") \
            .agg(count('postal_code').alias('postal_codes'),
                 max('postal_code').alias('postal_code')) \
            .orderBy('city', ascending=True) \
            .where("postal_codes=1")


cities_with_unique_postal_codes.show(5)   


+----------------+------------+-----------+
|            city|postal_codes|postal_code|
+----------------+------------+-----------+
|          Abades|           1|      40141|
|Abarca de Campos|           1|      34338|
|          Abejar|           1|      42146|
|         Abusejo|           1|      37640|
|  Adrada de Haza|           1|      09462|
+----------------+------------+-----------+
only showing top 5 rows



In [39]:
# Completing postalcodeless garitos with postal code when there is only one / city

sparkdf_garitos_pc_fixed = sparkdf_garitos_pc_null.join(
    cities_with_unique_postal_codes,
    sparkdf_garitos_pc_null.city == cities_with_unique_postal_codes.city,
    'left').select(
        'name',
        'address',
        'county',
        sparkdf_garitos_pc_null.city,
        cities_with_unique_postal_codes.postal_code,
        'garito_kind'
    )

print(sparkdf_garitos_pc_fixed.show(15))
# (It seems we were just able to fix 1)


+--------------------+--------------------+---------+--------------------+-----------+-----------+
|                name|             address|   county|                city|postal_code|garito_kind|
+--------------------+--------------------+---------+--------------------+-----------+-----------+
|           PUB EBANO|C/ JUAN FERRERO N...|     León|         Valderrueda|       null|      bares|
|              SILVAN|                null|     León|    Torre del Bierzo|       null|      bares|
|              FARINA|C/ ANTONIO ALMARZ...|    Ávila|               Ávila|       null|      bares|
|              AGUEDA|CAMINO DEL PEREGR...|     León|                León|       null|      bares|
|               VIFER| MAESTRO URIARTE, 25|     León|                León|       null|      bares|
|           LA CHISPA|C/ OBISPO MONTOYA...| Palencia|             Grijota|      34192|      bares|
|              RAMSES|CARRETERA ESTACIO...|    Ávila|         Sanchidrián|       null|      bares|
|         

In [40]:
sparkdf_garitos = (
        sparkdf_garitos.union(sparkdf_garitos_pc_fixed)
    )

### 2.2.4.- Garitos table. Example Queries.

In [7]:
# I want to practice with both dataframes and sparkdfs
# df shows here that there are addressless and postal_codeless
# garitos (garito= bar | restaurant | cafe) but no countyless or cityless
df=sparkdf_garitos.toPandas()
df.describe(include='all')

,name,address,county,city,postal_code,garito_kind
count,22487,22426,22487,22487,22472,22487
unique,16100,21087,9,1730,2025,3
top,LA PLAZA,"PLAZA MAYOR, 2",León,Valladolid,24003,bares
freq,87,20,4942,2668,369,15080


In [43]:
# Playing with Spark Data Frames. Most repeated names.
garitos_name_top = sparkdf_garitos \
    .select("name",'address') \
    .groupBy("name") \
    .agg(count("address").alias("Total")) \
    .orderBy("Total", ascending=False)
garitos_name_top.head(30)


[Row(name='LA PLAZA', Total=87),
 Row(name='AVENIDA', Total=55),
 Row(name='PLAZA', Total=47),
 Row(name='CENTRAL', Total=43),
 Row(name='TELEPIZZA', Total=41),
 Row(name='EL PASO', Total=36),
 Row(name='PISCINAS MUNICIPALES', Total=35),
 Row(name='LA TABERNA', Total=32),
 Row(name='EL RINCON', Total=32),
 Row(name='LA TERRAZA', Total=31),
 Row(name='BURGER KING', Total=31),
 Row(name='CASTILLA', Total=31),
 Row(name='EL CRUCE', Total=30),
 Row(name='LOS ARCOS', Total=30),
 Row(name='LA PARADA', Total=29),
 Row(name='EL PUENTE', Total=28),
 Row(name='LA BODEGUILLA', Total=27),
 Row(name='LA FUENTE', Total=27),
 Row(name='LOS ANGELES', Total=23),
 Row(name='EL MOLINO', Total=23),
 Row(name='EL PARQUE', Total=22),
 Row(name='MANOLO', Total=22),
 Row(name='LAS PISCINAS', Total=21),
 Row(name='LA CASONA', Total=21),
 Row(name='PISCINA MUNICIPAL', Total=21),
 Row(name='LA BODEGA', Total=21),
 Row(name='EL CASTILLO', Total=21),
 Row(name='EL REFUGIO', Total=20),
 Row(name='LA POSADA', Total=

In [5]:
# Playing with Spark Data Frames. 
# Most repeated restaurant names

restaurante_name_top = sparkdf_garitos \
    .select("name",'address','garito_kind') \
    .where("garito_kind='restaurantes'") \
    .groupBy("name",) \
    .agg(count("address").alias("Total")) \
    .orderBy("Total", ascending=False)

print(restaurante_name_top.head(30))



[Row(name='BURGER KING', Total=27), Row(name='TELEPIZZA', Total=18), Row(name='LA POSADA', Total=12), Row(name='EL MOLINO', Total=10), Row(name='LA TABERNA', Total=10), Row(name='AVENIDA', Total=10), Row(name='LA CASONA', Total=10), Row(name="FOSTER'S HOLLYWOOD", Total=9), Row(name='EL CRUCE', Total=8), Row(name='LOS ARCOS', Total=7), Row(name='BURGUER KING', Total=7), Row(name='CASTILLA', Total=7), Row(name='PLAZA', Total=7), Row(name="DOMINO'S PIZZA", Total=7), Row(name="MC DONALD'S", Total=7), Row(name='EL CASTILLO', Total=7), Row(name='LA PARADA', Total=7), Row(name='LA TERRAZA', Total=7), Row(name='CASA PACO', Total=6), Row(name='EL JARDIN', Total=6), Row(name='EL PASO', Total=6), Row(name='LA GRAN MURALLA', Total=6), Row(name='LA ENCINA', Total=6), Row(name='CENTRAL', Total=6), Row(name='LAS NIEVES', Total=6), Row(name='EL CAPRICHO', Total=6), Row(name='LA MURALLA', Total=6), Row(name='EL MESON', Total=6), Row(name='EL MIRADOR', Total=6), Row(name='HOGAR TERCERA EDAD', Total=5)]


In [51]:
# Playing with Spark Data Frames. 
# Counties order by number of cafeterias 

cafe_name_top = sparkdf_garitos \
    .select("county",'address') \
    .where("garito_kind='cafeterias'") \
    .groupBy("county",) \
    .agg(count("address").alias("Total")) \
    .orderBy("Total", ascending=False)

print(cafe_name_top.head(30))

[Row(county='León', Total=317), Row(county='Salamanca', Total=315), Row(county='Valladolid', Total=230), Row(county='Burgos', Total=179), Row(county='Ávila', Total=141), Row(county='Zamora', Total=91), Row(county='Segovia', Total=59), Row(county='Soria', Total=59), Row(county='Palencia', Total=54)]


In [54]:
# Playing with Spark Data Frames. 
# Order cities in a county (Burgos) by numero of garitos, but showing each type
burgos_top = sparkdf_garitos \
    .select("city",'address',
           when(sparkdf_garitos['garito_kind'] == 'cafeterias', 1).alias("is_cafe"),
           when(sparkdf_garitos['garito_kind'] == 'bares', 1).alias("is_bar"),
           when(sparkdf_garitos['garito_kind'] == 'restaurantes', 1).alias("is_restaurante")
           ) \
    .where("county='Burgos'") \
    .groupBy("city") \
    .agg(count("is_cafe").alias("cafes"), 
         count("is_bar").alias("bars"),
         count("is_restaurante").alias("restaurants"),
         count("address").alias("total"),
        ) \
    .orderBy("Total", ascending=False)


burgos_top.head(30)

[Row(city='Burgos', cafes=102, bars=765, restaurants=244, total=1110),
 Row(city='Aranda de Duero', cafes=19, bars=168, restaurants=62, total=249),
 Row(city='Miranda de Ebro', cafes=14, bars=177, restaurants=42, total=233),
 Row(city='Medina de Pomar', cafes=3, bars=67, restaurants=20, total=90),
 Row(city='Villarcayo de Merindad de Castilla la Vieja', cafes=7, bars=42, restaurants=16, total=65),
 Row(city='Briviesca', cafes=3, bars=42, restaurants=13, total=58),
 Row(city='Lerma', cafes=2, bars=19, restaurants=22, total=43),
 Row(city='Valle de Mena', cafes=2, bars=29, restaurants=13, total=41),
 Row(city='Espinosa de los Monteros', cafes=0, bars=25, restaurants=11, total=36),
 Row(city='Salas de los Infantes', cafes=0, bars=20, restaurants=9, total=29),
 Row(city='Roa', cafes=1, bars=18, restaurants=6, total=25),
 Row(city='Belorado', cafes=3, bars=12, restaurants=10, total=25),
 Row(city='Quintanar de la Sierra', cafes=0, bars=18, restaurants=7, total=24),
 Row(city='Melgar de Fern

### 2.2.5.- Garitos table --> Parquet TABLE.

In [19]:
## CLEANED UP GARITOS TO PARQUET 
toparquet_by_county_and_postcode(spark_session,local_parquet_path  + 'garitos/',sparkdf_garitos)


# 4.- SOCIAL TABLE
Social comprise any kind of associations, this including the special kind called sports associations (which information is in a different table / csv file). 

## 4.1.- Gather associations and sport clubs tables and puting them together into a SparkDF for Social.
These 2 files share a similar schema

In [8]:
## One from s3 (To test) ant the other from local folder

sparkdf_social=create_social(spark_session,local_path,data_asociaciones, data_clubes_deportivos) 

sparkdf_social.head(5)


social Spark Data Frame was created; 
  [Row(name='ASOCIACION DE JUBILADOS Y PENSIONISTAS VIRGEN DE LA PIEDAD', address='c/ Las Parras, 89', county='avila', city='EL BARRACO', postal_code='05110', sports='N/A', social_kind='association'), Row(name='PEÑA SAN MARCOS ', address='DE LAS FUENTES 10 ', county='avila', city='EL BARRACO', postal_code='05000', sports='N/A', social_kind='association')]


[Row(name='ASOCIACION DE JUBILADOS Y PENSIONISTAS VIRGEN DE LA PIEDAD', address='c/ Las Parras, 89', county='avila', city='EL BARRACO', postal_code='05110', sports='N/A', social_kind='association'),
 Row(name='PEÑA SAN MARCOS ', address='DE LAS FUENTES 10 ', county='avila', city='EL BARRACO', postal_code='05000', sports='N/A', social_kind='association'),
 Row(name='ASOCIACION DE JUBILADOS Y PENSIONISTAS DE SAN JUAN DEL OLMO SAN JUAN BAUTISTA', address='C/General Franco s/nº', county='avila', city='SAN JUAN DEL OLMO', postal_code='05145', sports='N/A', social_kind='association'),
 Row(name='ASOCIACION DEPORTIVO CULTURAL DE CAZADORES LA PICOTA', address='C/Piñonera nº 18', county='avila', city='CEBREROS', postal_code='05260', sports='N/A', social_kind='association'),
 Row(name='ASOCIACION CULTURAL -FORO 93-', address='AVDA. DE JOSE ANTONIO, 32 - 6', county='avila', city='EL HOYO DE PINARES', postal_code='05250', sports='N/A', social_kind='association')]

## 4.2.- Social table Quality Check and Wrangle/Cleanup

### 4.2.1.- Social table. Check for null values in key columns

In [135]:

sparkdf_museos = spark_session.read.options(multiLine=True).json(local_path+data_museos)

exp_df = sparkdf_museos.select('document.list.element.attribute')
exp_df2 = exp_df.select(monotonically_increasing_id().alias('library_id'), explode_outer('attribute').alias('next_evolution'))
exp_df3 = exp_df2.select(col('library_id'),explode('next_evolution').alias('next_evolution'))

fields=['Identificador','NombreOrganismo','Calle','CodigoPostal','Localidad_NombreLocalidad','Directorio Superior']

exp_df4=exp_df3.select(
    col('library_id'),
    col('next_evolution.name').alias('name'), 
    coalesce(col('next_evolution.valor'),
             col('next_evolution.LocalidadPadre'),
             col('next_evolution.string'),
             col('next_evolution.text')).alias('value'), 
    ).filter(col('name').isin(fields) == True)


exp_df5=exp_df4.select('*').groupBy("library_id").pivot('name').agg(collect_set('value')[0])

exp_df6=exp_df5.select(
    col('NombreOrganismo').alias('name'),
    col('Calle').alias('address'),
    col('Directorio Superior').alias('county'),
    col('Localidad_NombreLocalidad').alias('city'),
    col('CodigoPostal').alias('postal_code')
).withColumn("cultural_kind",lit('library')).distinct()

exp_df6.show(10)

+--------------------+--------------------+----------+-------------------+-----------+-------------+
|                name|             address|    county|               city|postal_code|cultural_kind|
+--------------------+--------------------+----------+-------------------+-----------+-------------+
|Museo Etnográfico...|Plaza de la Cruz, 4.|  Palencia|Cervera de Pisuerga|      34840|      library|
|Museo Pedagógico ...|    Plaza España, 7.|Valladolid|         Valladolid|      47001|      library|
|Espacio Informati...|C/ Martínez Campo...|   Segovia|            Segovia|      40001|      library|
|Domus Artium 2002...|Avda. Aldehuela s/n.| Salamanca|          Salamanca|      37003|      library|
|Centro de Interpr...|     Plaza Mayor, 1.|Valladolid|             Urueña|      47862|      library|
|Museo del Monaste...|Monasterio de Mad...|    Burgos|          Caleruega|      09451|      library|
|Martinete de la F...|        C/ Mayo, 15.|   Segovia|           Navafría|      40161|     

In [9]:
##CHECK FOR NULL VALUES IN KEY COLUMNS
check_sparkdf_not_nulls(sparkdf_social,['name','county','city','social_kind'])

Checking DataFrame. No null values found in column name
Checking DataFrame. No null values found in column county
Checking DataFrame. No null values found in column city
Checking DataFrame. No null values found in column social_kind


True

### 4.2.2.-Social table. Check for duplicate values and WRANGLE result.

In [10]:
##CHECK FOR REPEATED VALUES IN KEY COLUMNS
# We can accept same name, even same name in same city (if it is a different kind
# of association), but not same name / same city / same kind
dupes=check_sparkdf_find_dupes(sparkdf_social,['name','county','city','social_kind'])

print(dupes.head(15))

[Row(name='ASOCIACION JUVENIL CULTURAL 15 DE AGOSTO ', county='segovia', city='MOZONCILLO', social_kind='association', count=2), Row(name='ASOCIACIÓN CÍRCULO PSICOANALÍTICO DE LEÓN', county='león', city='LEÓN', social_kind='association', count=2), Row(name='ASOCIACION JUVENIL EL RIO ', county='segovia', city='CARRASCAL DEL RÍO', social_kind='association', count=2), Row(name='ANULADO', county='zamora', city='ZAMORA', social_kind='sports_club', count=2), Row(name='ASOCIACIÓN CULTURAL LUCERNA', county='león', city='CARUCEDO', social_kind='association', count=2)]


In [11]:
## We see there are several duplicates, one of them name="ANULADO" (Means voided)
## makes us think we should find other ANULADO/VOIDED and remove all of them

sparkdf_social_voided = sparkdf_social.select('*').where("name='ANULADO'").show()

+-------+--------------------+----------+--------------------+-----------+--------------------+-----------+
|   name|             address|    county|                city|postal_code|              sports|social_kind|
+-------+--------------------+----------+--------------------+-----------+--------------------+-----------+
|ANULADO|                 XXX|valladolid|          VALLADOLID|      47000|FU001#FÚTBOL - FÚ...|sports_club|
|ANULADO|C/ CORUÑA DEL CON...|    burgos|     ARANDA DE DUERO|       9400|FU001#FÚTBOL - FÚ...|sports_club|
|ANULADO|                JJJJ|  palencia|CAMPORREDONDO DE ...|      34888|CI003#CICLISMO - ...|sports_club|
|ANULADO|      Avda principal|     ávila|NAVAS DEL MARQUES...|       5230|CA001#CAZA - PICH...|sports_club|
|ANULADO|           XXXXXXXXX| salamanca|           SALAMANCA|      37008|MT018#MOTOCICLISM...|sports_club|
|ANULADO|C/ CARTAJENA DE I...|    zamora| MORALES DE VALVERDE|      49697|CA001#CAZA - PICH...|sports_club|
|ANULADO|    XXXXXXXXXXXXXXX

In [12]:
# changing name to ANULADO instead of removing the item seems a common practice in the sports_club 
# datasource. We get rid of all of them

sparkdf_social = sparkdf_social.select('*').where("name <> 'ANULADO'")

In [13]:
# Also, we want to remove other duplicates

print('Rows before removing duplicates',sparkdf_social.count())
sparkdf_social=sparkdf_social.dropDuplicates(['name','county','city','social_kind'])
print('Rows after removing duplicates',sparkdf_social.count())


Rows before removing duplicates 43738
Rows after removing duplicates 43734


### 4.2.3.- Social table Wrangling: check counties that are not valid. Guess those that can be guessed (soft typos) and remove the others.
Note: There are some cases of wrong counties containing special characters (like Ñ) that spoils out the parquet table creation as it would end in a hardly descriptive error. 

In [14]:
# CHECK and remove wrong Counties as we are going to partition by county
print('Rows before removing duplicates',sparkdf_social.count())
sparkdf_social=clean_wrong_counties(sparkdf_social)
print('Rows after removing duplicates',sparkdf_social.count())


sparkdf_social.select("County").distinct().head(80)


[Row(County='león'),
 Row(County='ávila'),
 Row(County='segovia'),
 Row(County='palencia'),
 Row(County='soria'),
 Row(County='burgos'),
 Row(County='zamora'),
 Row(County='valladolid'),
 Row(County='salamanca')]

### 4.2.4.- Social table --> Parquet TABLE.

In [11]:

## CLEANED UP SOCIAL TO PARQUET (LOCAL)
toparquet_by_county_and_postcode(spark_session,local_parquet_path  + 'social/',sparkdf_social)

In [32]:
## CLEANED UP SOCIAL TO PARQUET (S3)

#bucket_name = 'raul-udacity'
#bucket_parquet_path ='/parquet/'
#Sparkdf_social.write.partitionBy("county","postal_code").parquet('s3a://'+bucket_name+bucket_parquet_path + "social/", mode="overwrite")

### 4.2.5.- Example queries to SOCIAL

In [17]:
## Sports / county
# https://stackoverflow.com/questions/57066797/pyspark-dataframe-split-column-with-multiple-values-into-rows#57080133

from pyspark.sql.functions import explode, regexp_replace, split

out=sparkdf_social.withColumn(
    "sports", 
    explode(split(col("sports"), "\|"))
).where("county='burgos'").select(
    col('sports')
    ).groupBy('sports').agg(count('sports').alias('sport_associations')) \
    .orderBy('sport_associations', ascending=False)



out.head(70)

[Row(sports='N/A', sport_associations=5423),
 Row(sports='', sport_associations=1102),
 Row(sports='CA005#CAZA - PERROS DE CAZA Y AGILITY#', sport_associations=385),
 Row(sports='CA013#CAZA - EDUCACIÓN CANINA#', sport_associations=385),
 Row(sports='CA006#CAZA - CETRERIA#', sport_associations=384),
 Row(sports='CA009#CAZA - TIRO A CAZA LANZADA#', sport_associations=384),
 Row(sports='CA001#CAZA - PICHON A BRAZO#', sport_associations=384),
 Row(sports='CA012#CAZA - PERDIZ CON RECLAMO#', sport_associations=384),
 Row(sports='CA014#CAZA - CAZA DE BECADAS#', sport_associations=384),
 Row(sports='CA008#CAZA - CAZA CON ARCO#', sport_associations=384),
 Row(sports='CA010#CAZA - CAZA FOTOGRAFICA Y VIDEO#', sport_associations=384),
 Row(sports='CA011#CAZA - COMPAK SPORTING#', sport_associations=384),
 Row(sports='CA004#CAZA - CAZA SAN HUBERTO#', sport_associations=384),
 Row(sports='CA007#CAZA - PAJAROS DE CANTO#', sport_associations=383),
 Row(sports='CA003#CAZA - RECORRIDOS DE CAZA#', sport_a

# 5.- CULTURAL TABLE

Cultural comprise libraries (bibliotecas), museums (museos) and it could also include more cases in the future. 
To mention that these tables inputs **are in JSON**

## 5.1.- Gather libraries (bibliotecas), museums (museos) and puting them together into a SparkDF for Cultural.

These 2 files share a similar schema


In [5]:
sparkdf_cultural=create_cultural_from_json(spark_session,local_path,[data_bibliotecas,data_museos])

In [5]:
sparkdf_cultural.head(10)


[Row(name='Biblioteca Pública Municipal de Valverde del Majano', address='C/ Real, 10', county='Segovia', city='Valverde del Majano', postal_code='40140', cultural_kind='Directorio de Bibliotecas de Castilla y León'),
 Row(name="Biblioteca Pública Municipal del Centro Cívico 'Río Vena' de Burgos", address='C/ Juan de Padilla, s/n - Centro Cívico, 1ª planta', county='Burgos', city='Burgos', postal_code='09006', cultural_kind='Directorio de Bibliotecas de Castilla y León'),
 Row(name='Biblioteca Pública Municipal de Villanueva del Campo', address='C/ Federico Arrazola, s/n', county='Zamora', city='Villanueva del Campo', postal_code='49100', cultural_kind='Directorio de Bibliotecas de Castilla y León'),
 Row(name='Centro de Documentación Europea. Valladolid', address='Pza. de Sta. Cruz, 5, 3ª planta', county='Valladolid', city='Valladolid', postal_code='47002', cultural_kind='Directorio de Bibliotecas de Castilla y León'),
 Row(name='Biblioteca de la Universidad de León', address='Campus 

In [6]:
sparkdf_cultural.describe()


DataFrame[summary: string, name: string, address: string, county: string, city: string, postal_code: string, cultural_kind: string]

## 5.2.- Cultural table Quality Checks and Wrangle/Cleanup

### 5.2.1.- Cultura table. Check for null values in key columns

In [7]:
##CHECK FOR NULL VALUES IN KEY COLUMNS
check_sparkdf_not_nulls(sparkdf_cultural,['name','county','city','cultural_kind'])

Checking DataFrame. No null values found in column name
Checking DataFrame. No null values found in column county
Checking DataFrame. No null values found in column city
Checking DataFrame. No null values found in column cultural_kind


True

### 5.2.2.-Cultural table. Check for duplicate values and WRANGLE result.

In [8]:
##CHECK FOR REPEATED VALUES IN KEY COLUMNS
# We can accept same name, even same name in same city (if it is a different kind
# of association), but not same name / same city / same kind
dupes=check_sparkdf_find_dupes(sparkdf_cultural,['name','county','city','cultural_kind'])

print(dupes.head(15))

[Row(name='Biblioteca Pública Municipal de Ávila', county='Ávila', city='Ávila', cultural_kind='Directorio de Bibliotecas de Castilla y León', count=3), Row(name='Museo de la Industria Harinera de Castilla y León (MIHACALE)', county='León', city='Gordoncillo', cultural_kind='Directorio de Museos de Castilla y León', count=2)]


In [9]:
# In this case, they are city libraries with the same name and in the same city, but different address 
# meaning the library has 3 buildings / offices
# So I decide not to remove duplicates (Address is different)

dupes=check_sparkdf_find_dupes(sparkdf_cultural,['name','address','county','city','cultural_kind'])
print(dupes.head(15))

[]


### 5.2.3.- Cultural table Wrangling: check counties that are not valid. Guess those that can be guessed (soft typos) and remove the others.
Note: There are some cases of wrong counties containing special characters (like Ñ) that spoils out the parquet table creation as it would end in a hardly descriptive error. 

In [10]:
# CHECK and remove wrong Counties as we are going to partition by county
print('Rows before removing duplicates',sparkdf_cultural.count())
sparkdf_cultural=clean_wrong_counties(sparkdf_cultural)
print('Rows after removing duplicates',sparkdf_cultural.count())
sparkdf_cultural.head(3)

Rows before removing duplicates 828
Rows after removing duplicates 828


[Row(name='Biblioteca Pública Municipal de Valverde del Majano', address='C/ Real, 10', county='segovia', city='Valverde del Majano', postal_code='40140', cultural_kind='Directorio de Bibliotecas de Castilla y León'),
 Row(name="Biblioteca Pública Municipal del Centro Cívico 'Río Vena' de Burgos", address='C/ Juan de Padilla, s/n - Centro Cívico, 1ª planta', county='burgos', city='Burgos', postal_code='09006', cultural_kind='Directorio de Bibliotecas de Castilla y León'),
 Row(name='Biblioteca Pública Municipal de Villanueva del Campo', address='C/ Federico Arrazola, s/n', county='zamora', city='Villanueva del Campo', postal_code='49100', cultural_kind='Directorio de Bibliotecas de Castilla y León')]

### 5.2.4.- Cultural table --> Parquet TABLE.

In [9]:
## CLEANED UP CULTURAL TO PARQUET (LOCAL)
toparquet_by_county_and_postcode(spark_session,local_parquet_path  + 'cultural/',sparkdf_cultural)

### 5.2.5.- Example queries to CULTURAL

In [22]:
# Order cities by number of cultural items (libraries + museums)
cultural_top = sparkdf_cultural \
    .select('county','city','address',
           when(sparkdf_cultural['cultural_kind'] == 'Directorio de Museos de Castilla y León', 1).alias("is_museum"),
           when(sparkdf_cultural['cultural_kind'] == 'Directorio de Bibliotecas de Castilla y León', 1).alias("is_library")
    ) \
    .groupBy('county','city') \
    .agg(
        count("is_museum").alias("museums"), 
        count("is_library").alias("libraries"),
        count("address").alias("total")
        ) \
    .orderBy("Total", ascending=False)

    

cultural_top.head(30)

[Row(county='valladolid', city='Valladolid', museums=25, libraries=36, total=61),
 Row(county='burgos', city='Burgos', museums=11, libraries=26, total=37),
 Row(county='león', city='León', museums=13, libraries=14, total=27),
 Row(county='salamanca', city='Salamanca', museums=17, libraries=10, total=27),
 Row(county='ávila', city='Ávila', museums=12, libraries=10, total=22),
 Row(county='zamora', city='Zamora', museums=8, libraries=12, total=20),
 Row(county='segovia', city='Segovia', museums=13, libraries=6, total=19),
 Row(county='soria', city='Soria', museums=5, libraries=7, total=12),
 Row(county='palencia', city='Palencia', museums=7, libraries=5, total=12),
 Row(county='león', city='Ponferrada', museums=6, libraries=2, total=8),
 Row(county='valladolid', city='Medina del Campo', museums=6, libraries=2, total=8),
 Row(county='valladolid', city='Medina de Rioseco', museums=7, libraries=1, total=8),
 Row(county='valladolid', city='Tordesillas', museums=6, libraries=1, total=7),
 Row

In [24]:
# List museums in Aranda de Duero
museums_in_Aranda = sparkdf_cultural \
    .select('name','address') \
    .where("city='Aranda de Duero'").where("cultural_kind like '%Museo%'")

    

museums_in_Aranda.head(5)

[Row(name='Museo de Alfarería y Cerámica', address='C/ Don Julián y Don Romero, s/n.'),
 Row(name='Museo del Tren', address='Estación Chelva, s/n.'),
 Row(name='Museo Casa de las Bolas. Colección Permanente Félix Cañada', address='Casa de las Bolas. Plaza de San Juan, 9.'),
 Row(name='Museo Sacro de Aranda de Duero', address='Iglesia de San Juan, C/ San Juan, s/n.'),
 Row(name='Centro de Interpretación de la Arquitectura Asociada al Vino', address='Plaza Mayor, s/n.')]

In [41]:
# List museums having to do with wine (vino in Spanish) or Cheese (queso in Spanish)
wine_museums = sparkdf_cultural \
    .select('county','city', 'name', 'address') \
    .where("cultural_kind like '%Museo%'").where(
    "name like '%Vino%' or name like '%vino%' or name like '%ueso%'"
    ).orderBy('county','city')

    

wine_museums.head(15)

[Row(county='burgos', city='Aranda de Duero', name='Centro de Interpretación de la Arquitectura Asociada al Vino', address='Plaza Mayor, s/n.'),
 Row(county='león', city='Cacabelos', name='Museo Vinos del Bierzo', address='Avda. Constitución , 106.'),
 Row(county='palencia', city='Frómista', name='Museo del Queso', address='Plaza de San Martín, 8'),
 Row(county='valladolid', city='Peñafiel', name='Museo Provincial del Vino', address='Castillo.'),
 Row(county='valladolid', city='Villalón de Campos', name='Museo del Queso de Villalón de Campos', address='Avenida del Parque, 7.'),
 Row(county='zamora', city='Morales de Toro', name='Museo del Vino Pagos del Rey', address='Avda/ de los Comuneros, 90'),
 Row(county='zamora', city='Toro', name='Museo del Queso Chillón', address='Carretera Tordesillas 11')]

# 6.- POPULATION TABLE

Cities population per gender age.csv contains population per gender age and city in Castilla y León
We'll group it buckets and assign county to any city (There is no county field)


In [133]:
	sparkdf_handle = spark_session.read.options(inferSchema='true',\
					delimiter=';',
					header='true',
					encoding='ISO-8859-1').csv(local_path+data_poblacion)


In [49]:
# first we filter out municipalities that are not in Castilla y León region
# We can use the Municipios code (starts by the postal code of the cities)
# We convert Total to number and '..' into 0 

[Row(Municipios='TOTAL NACIONAL', Sexo='Ambos sexos', Edad (aÃ±o a aÃ±o)='Total', Total='46.815.916'),
 Row(Municipios='TOTAL NACIONAL', Sexo='Ambos sexos', Edad (aÃ±o a aÃ±o)='0 aÃ±os', Total='480.267'),
 Row(Municipios='TOTAL NACIONAL', Sexo='Ambos sexos', Edad (aÃ±o a aÃ±o)='1 aÃ±o', Total='481.568'),
 Row(Municipios='TOTAL NACIONAL', Sexo='Ambos sexos', Edad (aÃ±o a aÃ±o)='2 aÃ±os', Total='497.738'),
 Row(Municipios='TOTAL NACIONAL', Sexo='Ambos sexos', Edad (aÃ±o a aÃ±o)='3 aÃ±os', Total='519.804'),
 Row(Municipios='TOTAL NACIONAL', Sexo='Ambos sexos', Edad (aÃ±o a aÃ±o)='4 aÃ±os', Total='500.817'),
 Row(Municipios='TOTAL NACIONAL', Sexo='Ambos sexos', Edad (aÃ±o a aÃ±o)='5 aÃ±os', Total='498.131'),
 Row(Municipios='TOTAL NACIONAL', Sexo='Ambos sexos', Edad (aÃ±o a aÃ±o)='6 aÃ±os', Total='487.253'),
 Row(Municipios='TOTAL NACIONAL', Sexo='Ambos sexos', Edad (aÃ±o a aÃ±o)='7 aÃ±os', Total='478.292'),
 Row(Municipios='TOTAL NACIONAL', Sexo='Ambos sexos', Edad (aÃ±o a aÃ±o)='8 aÃ±os'

In [171]:
from pyspark.sql.functions import regexp_replace, regexp_extract, expr
from pyspark.sql.types import IntegerType

print(sparkdf_handle.count())

sparkdf_handle2=sparkdf_handle \
    .select(col('Municipios').alias('city'),
            col('Edad (aÃ±o a aÃ±o)').alias('age'),
            col('Total').alias('population'))\
    .where("Municipios like '05%' or Municipios like '09%' or Municipios like '24%' or \
           Municipios like '34%' or Municipios like '37%' or Municipios like '40%' or \
           Municipios like '42%' or Municipios like '47%' or Municipios like '49%' ") \
    .where("Sexo ='Ambos sexos'")\
    .where("age <> 'Total'")\
    .withColumn('population', regexp_replace('population','\.\.','0')) \
    .withColumn('population', regexp_replace('population','\.','')) \
    .withColumn('age', regexp_extract('age', '([0-9]*)(\s+)', 1)) \
    .withColumn("county",expr("case when city like '05%' then 'ávila' " +
                       "when city like '09%' then 'burgos' " +
                              "when city like '24%' then 'león' " +
                              "when city like '34%' then 'palencia' " +
                              "when city like '37%' then 'salamanca' " +
                              "when city like '40%' then 'segovia' " +
                              "when city like '42%' then 'soria' " +
                              "when city like '47%' then 'valladolid' " +
                              "when city like '49%' then 'zamora' " +
                       "else 'Great' end")) \
    .withColumn('city', regexp_extract('city', '([0-9]*)(\s+)(.+)', 3)) \
    .withColumn('city', regexp_replace('city', 'Ã¡', 'á')) \
    .withColumn('city', regexp_replace('city', 'Ã\x81', 'Á')) \
    .withColumn('city', regexp_replace('city', 'Ã©', 'é')) \
    .withColumn('city', regexp_replace('city', 'Ã\xad', 'í')) \
    .withColumn('city', regexp_replace('city', 'Ã\x8d', 'Í')) \
    .withColumn('city', regexp_replace('city', 'Ã³', 'ó')) \
    .withColumn('city', regexp_replace('city', 'Ã\x93', 'Ó')) \
    .withColumn('city', regexp_replace('city', 'Ã\x9a', 'Ú')) \
    .withColumn('city', regexp_replace('city', 'Ãº', 'ú')) \
    .withColumn('city', regexp_replace('city', 'Ã¼', 'ü')) \
    .withColumn('city', regexp_replace('city', 'Ã±', 'ñ')) \




print(sparkdf_handle2.count())
print(sparkdf_handle2.describe())


2483802
227048
DataFrame[summary: string, city: string, age: string, population: string, county: string]


In [172]:
sparkdf_handle3 = sparkdf_handle2. \
    withColumn('age', col('age').cast(IntegerType())). \
    withColumn('population', col('population').cast(IntegerType()))


sparkdf_handle3.printSchema()

root
 |-- city: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- population: integer (nullable = true)
 |-- county: string (nullable = false)



In [174]:
sparkdf_handle4 = sparkdf_handle3.select('county','city','age','population')\
        .withColumn('under16',expr('case when age <16 then population else 0 end'))\
        .withColumn('16to30',expr('case when age >= 16 and age <31 then population else 0 end'))\
        .withColumn('31to45',expr('case when age >= 31 and age <46 then population else 0 end'))\
        .withColumn('46to65',expr('case when age >= 46 and age <66 then population else 0 end'))\
        .withColumn('66to80',expr('case when age >= 66 and age <81 then population else 0 end'))\
        .withColumn('over80',expr('case when age >= 81 then population else 0 end'))\
        .groupBy('county','city') \
        .agg({'under16': 'sum' , '16to30': 'sum' , '31to45': 'sum', '46to65': 'sum' , '66to80': 'sum' , 'over80': 'sum'} )\
        .withColumn("total Population",col('sum(under16)')+col('sum(16to30)')+col('sum(31to45)')
                   +col('sum(46to65)')+col('sum(66to80)')+col('sum(over80)'))
sparkdf_handle4.head(2000)

[Row(county='ávila', city='Adanero', sum(46to65)=57, sum(66to80)=51, sum(16to30)=26, sum(over80)=25, sum(31to45)=72, sum(under16)=36, total Population=267),
 Row(county='zamora', city='Alcañices', sum(46to65)=289, sum(66to80)=214, sum(16to30)=188, sum(over80)=123, sum(31to45)=242, sum(under16)=111, total Population=1167),
 Row(county='soria', city='Bayubas de Abajo', sum(46to65)=51, sum(66to80)=57, sum(16to30)=18, sum(over80)=35, sum(31to45)=31, sum(under16)=6, total Population=198),
 Row(county='salamanca', city='Cantalpino', sum(46to65)=255, sum(66to80)=211, sum(16to30)=129, sum(over80)=99, sum(31to45)=207, sum(under16)=98, total Population=999),
 Row(county='salamanca', city='Cerralbo', sum(46to65)=50, sum(66to80)=65, sum(16to30)=17, sum(over80)=39, sum(31to45)=21, sum(under16)=4, total Population=196),
 Row(county='palencia', city='Dueñas', sum(46to65)=866, sum(66to80)=365, sum(16to30)=462, sum(over80)=189, sum(31to45)=606, sum(under16)=315, total Population=2803),
 Row(county='leó

In [175]:
sparkdf_handle5 = sparkdf_handle4.select('*').orderBy('total Population', ascending=False)
    
sparkdf_handle5.head(1000)

[Row(county='valladolid', city='Valladolid', sum(46to65)=89635, sum(66to80)=45347, sum(16to30)=47341, sum(over80)=17027, sum(31to45)=71496, sum(under16)=40839, total Population=311685),
 Row(county='burgos', city='Burgos', sum(46to65)=48268, sum(66to80)=21576, sum(16to30)=30005, sum(over80)=10133, sum(31to45)=41954, sum(under16)=26924, total Population=178860),
 Row(county='salamanca', city='Salamanca', sum(46to65)=41114, sum(66to80)=23174, sum(16to30)=24529, sum(over80)=11166, sum(31to45)=31819, sum(under16)=19859, total Population=151661),
 Row(county='león', city='León', sum(46to65)=37143, sum(66to80)=19819, sum(16to30)=19932, sum(over80)=9698, sum(31to45)=28504, sum(under16)=16319, total Population=131415),
 Row(county='palencia', city='Palencia', sum(46to65)=23385, sum(66to80)=10387, sum(16to30)=13287, sum(over80)=5023, sum(31to45)=17953, sum(under16)=11055, total Population=81090),
 Row(county='león', city='Ponferrada', sum(46to65)=18600, sum(66to80)=9338, sum(16to30)=11264, sum(